XGB算法梳理

算法原理
损失函数
分裂结点算法
正则化
对缺失值处理
优缺点
应用场景
sklearn参数

1.传统GBDT以CART作为基分类器，xgboost还支持线性分类器，这个时候xgboost相当于带L1和L2正则化项的逻辑斯蒂回归（分类问题）或者线性回归（回归问题）。 —可以通过booster [default=gbtree]设置参数:gbtree: tree-based models/gblinear: linear models


--------------------- 
版权声明：本文为CSDN博主「我曾经被山河大海跨过」的原创文章，遵循CC 4.0 by-sa版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/sb19931201/article/details/52557382

2.传统GBDT在优化时只用到一阶导数信息，xgboost则对代价函数进行了二阶泰勒展开，同时用到了一阶和二阶导数。顺便提一下，xgboost工具支持自定义代价函数，只要函数可一阶和二阶求导。 —对损失函数做了改进（泰勒展开，一阶信息g和二阶信息h,上一章节有做介绍）

3 split finding algorithms(划分点查找算法)：—理解的还不够透彻，需要进一步学习 
（1）exact greedy algorithm—贪心算法获取最优切分点 
（2）approximate algorithm— 近似算法，提出了候选分割点概念，先通过直方图算法获得候选分割点的分布情况，然后根据候选分割点将连续的特征信息映射到不同的buckets中，并统计汇总信息。详细见论文3.3节 
（3）Weighted Quantile Sketch—分布式加权直方图算法，论文3.4节 
这里的算法（2）、（3）是为了解决数据无法一次载入内存或者在分布式情况下算法（1）效率低的问题，以下引用的还是wepon大神的总结：
--------------------- 
版权声明：本文为CSDN博主「我曾经被山河大海跨过」的原创文章，遵循CC 4.0 by-sa版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/sb19931201/article/details/52557382

4.xgboost在代价函数里加入了正则项，用于控制模型的复杂度。正则项里包含了树的叶子节点个数、每个叶子节点上输出的score的L2模的平方和。从Bias-variance tradeoff角度来讲，正则项降低了模型variance，使学习出来的模型更加简单，防止过拟合，这也是xgboost优于传统GBDT的一个特性 
--------------------- 
版权声明：本文为CSDN博主「我曾经被山河大海跨过」的原创文章，遵循CC 4.0 by-sa版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/sb19931201/article/details/52557382

5 6.对缺失值的处理。对于特征的值有缺失的样本，xgboost可以自动学习出它的分裂方向。 —稀疏感知算法，论文3.4节，Algorithm 3: Sparsity-aware Split Finding

6

7

8.General Parameters（常规参数） 
1.booster [default=gbtree]：选择基分类器，gbtree: tree-based models/gblinear: linear models 
2.silent [default=0]:设置成1则没有运行信息输出，最好是设置为0. 
3.nthread [default to maximum number of threads available if not set]：线程数

Booster Parameters（模型参数） 
1.eta [default=0.3]:shrinkage参数，用于更新叶子节点权重时，乘以该系数，避免步长过大。参数值越大，越可能无法收敛。把学习率 eta 设置的小一些，小学习率可以使得后面的学习更加仔细。 
2.min_child_weight [default=1]:这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
3.max_depth [default=6]: 每颗树的最大深度，树高越深，越容易过拟合。 
4.max_leaf_nodes:最大叶结点数，与max_depth作用有点重合。 
5.gamma [default=0]：后剪枝时，用于控制是否后剪枝的参数。 
6.max_delta_step [default=0]：这个参数在更新步骤中起作用，如果取0表示没有约束，如果取正值则使得更新步骤更加保守。可以防止做太大的更新步子，使更新更加平缓。 
7.subsample [default=1]：样本随机采样，较低的值使得算法更加保守，防止过拟合，但是太小的值也会造成欠拟合。 
8.colsample_bytree [default=1]：列采样，对每棵树的生成用的特征进行列采样.一般设置为： 0.5-1 
9.lambda [default=1]：控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。 
10.alpha [default=0]:控制模型复杂程度的权重值的 L1 正则项参数，参数值越大，模型越不容易过拟合。 
11.scale_pos_weight [default=1]：如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。

Learning Task Parameters（学习任务参数） 
1.objective [default=reg:linear]：定义最小化损失函数类型，常用参数： 
binary:logistic –logistic regression for binary classification, returns predicted probability (not class) 
multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities) 
you also need to set an additional num_class (number of classes) parameter defining the number of unique classes 
multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class. 
2.eval_metric [ default according to objective ]： 
The metric to be used for validation data. 
The default values are rmse for regression and error for classification. 
Typical values are: 
rmse – root mean square error 
mae – mean absolute error 
logloss – negative log-likelihood 
error – Binary classification error rate (0.5 threshold) 
merror – Multiclass classification error rate 
mlogloss – Multiclass logloss 
auc: Area under the curve 
3.seed [default=0]： 
The random number seed. 随机种子，用于产生可复现的结果 
Can be used for generating reproducible results and also for parameter tuning.

注意: python sklearn style参数名会有所变化 
eta –> learning_rate 
lambda –> reg_lambda 
alpha –> reg_alpha
--------------------- 
版权声明：本文为CSDN博主「我曾经被山河大海跨过」的原创文章，遵循CC 4.0 by-sa版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/sb19931201/article/details/52557382

In [8]:
from sklearn import svm
X = [[0, 0], [1, 1]]
y = [0, 1]
clf = svm.SVC()
clf.fit(X, y)  
clf.predict([[2., 2.]])

ImportError: DLL load failed: 找不到指定的模块。

In [6]:
from sklearn import svm

ImportError: DLL load failed: 找不到指定的模块。